In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, \
    DataCollatorWithPadding, pipeline
from datasets import load_dataset
import torch
import sklearn

In [23]:
dataset = load_dataset("csv", data_files="data/ChnSentiCorp_htl_all.csv", split="train")  # 只加载训练集
dataset = dataset.filter(lambda x: x["review"] is not None)  # 去除None
print(dataset)

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})


In [24]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

In [25]:
tokenizer = AutoTokenizer.from_pretrained('models/hfl/rbt3')
model = AutoModelForSequenceClassification.from_pretrained("models/hfl/rbt3")


def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], truncation=True, max_length=128)
    tokenized_examples['labels'] = examples["label"]
    return tokenized_examples


tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

/home/ubuntu/anaconda3/envs/torch/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at models/hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 777
    })
})

In [35]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    f1 = sklearn.metrics.f1_score(labels, predictions, average='macro')
    acc = sklearn.metrics.accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1}

In [41]:
output_dir = 'models/my_model/classification'
train_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=64,  # 训练时的batch_size
    per_device_eval_batch_size=128,  # 验证时的batch_size
    logging_steps=10,  # log 打印的频率
    save_strategy='no',
    eval_strategy="epoch",  # 评估策略
    learning_rate=2e-5,  # 学习率
    weight_decay=0.01,  # weight_decay
    metric_for_best_model="eval_loss",  # 设定评估指标
    greater_is_better=False,  # 损失值越小越好
)
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=eval_metric,
)

In [42]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.018000,0.503534,0.885457,0.867506
2,0.032100,0.496351,0.895753,0.875405
3,0.014800,0.496692,0.904762,0.887144


TrainOutput(global_step=330, training_loss=0.022912893026615633, metrics={'train_runtime': 54.8732, 'train_samples_per_second': 382.044, 'train_steps_per_second': 6.014, 'total_flos': 351909933963264.0, 'train_loss': 0.022912893026615633, 'epoch': 3.0})

In [43]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.4966924786567688,
 'eval_accuracy': 0.9047619047619048,
 'eval_f1': 0.8871437544162676,
 'eval_runtime': 0.7834,
 'eval_samples_per_second': 991.828,
 'eval_steps_per_second': 8.935,
 'epoch': 3.0}

In [44]:
trainer.predict(tokenized_datasets["test"])

PredictionOutput(predictions=array([[-4.873064 ,  4.6434507],
       [-4.935604 ,  5.0221043],
       [-4.4198728,  4.4736114],
       ...,
       [-4.835693 ,  4.7990355],
       [-2.175477 ,  2.976697 ],
       [ 5.2299685, -4.538448 ]], dtype=float32), label_ids=array([1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
    

In [45]:
id2_label = {0: "差评！", 1: "好评！"}
model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [54]:
sen1 = "我觉得不错！"
sen2 = "我觉得服务一般。"
sen3 = "他们家的卫生一团糟"
sen4 = "他们家的服务很好，但是卫生很差"
sen5 = "他们家的卫生很差，但是服务很好"
sen6 = "我觉得他们的食物不错"
s1, s2, s3, s4, s5, s6 = pipe(sen1), pipe(sen2), pipe(sen3), pipe(sen4), pipe(sen5), pipe(sen6)
print(s1, s2, s3, s4, s5, s6)

[{'label': '好评！', 'score': 0.999708354473114}] [{'label': '差评！', 'score': 0.5323366522789001}] [{'label': '差评！', 'score': 0.9966638684272766}] [{'label': '差评！', 'score': 0.9915008544921875}] [{'label': '好评！', 'score': 0.7534747123718262}] [{'label': '好评！', 'score': 0.999834418296814}]
